This notebook organizes all the results in the "analysis" folder within a concrete storyline

In [1]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di

# This line will hide code by default when the notebook is converted to HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

In [2]:
%load_ext autoreload
%autoreload 1
%aimport utils_read_parsing
from utils_read_parsing import *
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from tabulate import tabulate
import seaborn as sns

%matplotlib inline
matplotlib.style.use('ggplot')
matplotlib.rcParams['figure.figsize'] = 7, 4
pd.set_option('precision',3)
pd.set_option("display.width", 100)
np.set_printoptions(precision=3,suppress=True)

# Preparing the data
We wish to have a table that looks like:

In [3]:
t = [['sid','sim','variable','model','pre_statment','pre_correct','main_statement','main_correct','CVS_context','tansfer_quant','attributes']]
t.append(['12345678','C','Width','ident','1','1','0','NaN','2',1,'...'])
t.append(['12345678','C','Width','qual','0','NaN','1','0','2',0,'...'])
t.append(['12345678','C','Width','quant','0','NaN','0','NaN','2',0,'...'])
print tabulate(t)

--------  ---  --------  -----  ------------  -----------  --------------  ------------  -----------  -------------  ----------
sid       sim  variable  model  pre_statment  pre_correct  main_statement  main_correct  CVS_context  tansfer_quant  attributes
12345678  C    Width     ident  1             1            0               NaN           2            1              ...
12345678  C    Width     qual   0             NaN          1               0             2            0              ...
12345678  C    Width     quant  0             NaN          0               NaN           2            0              ...
--------  ---  --------  -----  ------------  -----------  --------------  ------------  -----------  -------------  ----------


## Defining important variables

In [4]:
ids = get_students_to_analyze_log()
N = len(ids)
print "There are {0} students to analyze.".format(N)

There are 148 students to analyze.


In [5]:
variables = ["Area","Separation","Width","Concentration"]
print "We are analyzing student knowledge and modeling of these variables: {0}.".format(', '.join(variables))

We are analyzing student knowledge and modeling of these variables: Area, Separation, Width, Concentration.


In [6]:
merged = True
number_of_trials = 3
if merged:
    print '''We will be analyzing cvs where table_consecutive and table_non_consecutive are merged
        and CVS is detected for {0} minimum trials.'''.format(number_of_trials)
else:
    print '''We will be analyzing cvs where table_consecutive and table_non_consecutive are NOT merged
        and CVS is detected for {0} minimum trials.'''.format(number_of_trials)

We will be analyzing cvs where table_consecutive and table_non_consecutive are merged
        and CVS is detected for 3 minimum trials.


## Loading all student data files

In [7]:
%reload_ext utils_read_parsing
meta_df = get_student_metadata()
pre_survey_df = get_massaged_pre_survey()
post_survey_df = get_massaged_post_survey()
attitudes_df = get_incoming_attitudes()
worksheet_df = get_massaged_worksheet_model_data()
cvs_df = get_cvs_results(number_of_trials,merged=merged)
transfer_df = get_massaged_near_transfer_data()
wrapper_use_df = get_use_wrapper_results().rename(columns={'studentid':'sid'})

In [8]:
#We only keep the worksheet data,CVS and transfer data for the variables we care about
worksheet_df = worksheet_df[worksheet_df['variable'].isin(variables)]
cvs_df = cvs_df[cvs_df['variable'].isin(variables)]
transfer_df = transfer_df[transfer_df['variable'].isin(variables)]

In [9]:
print "Here are the dataframes and their sizes:"
print "meta", meta_df.shape
print "pre_survey", pre_survey_df.shape
print "post_survey", post_survey_df.shape
print "worksheet", worksheet_df.shape
print "cvs", cvs_df.shape
print "transfer", transfer_df.shape
print "wrapper_use", wrapper_use_df.shape

Here are the dataframes and their sizes:
meta (180, 23)
pre_survey (148, 24)
post_survey (296, 70)
worksheet (1776, 8)
cvs (592, 3)
transfer (592, 4)
wrapper_use (148, 7)


## Putting it all together
The worksheet dataframe is the most complex with 1 row for each combination of sid * variable * model so we merge other data onto it.

In [10]:
data = worksheet_df.copy()
data = data.merge(cvs_df, how='outer', on=['sid','variable']);
data = data.merge(transfer_df, how='outer', on=['sid','variable']);
data = data.merge(wrapper_use_df, how='outer', on=['sid']);

In [11]:
## We add the student activity order
order = dict(zip(meta_df.index,meta_df['activity order']))
data['activity order'] = data.sid.apply(lambda sid: order[sid])

In [12]:
data.head()

,sid,sim,variable,model,pre_statement,pre_correct,main_statement,main_correct,CVS_context,qual_score,quant_score,use_table,use_graph,use_concentration,use_width,use_area,use_separation,activity order
0,10127163,C,Area,ident,0.0,NaN,0.0,NaN,2,1.0,1.0,1,1,1,1,1,1,LC
1,10127163,C,Area,qual,1.0,1.0,1.0,1.0,2,1.0,1.0,1,1,1,1,1,1,LC
2,10127163,C,Area,quant,1.0,1.0,1.0,1.0,2,1.0,1.0,1,1,1,1,1,1,LC
3,10127163,L,Concentration,ident,0.0,NaN,0.0,NaN,2,1.0,1.0,1,1,1,1,1,1,LC
4,10127163,L,Concentration,qual,1.0,0.0,1.0,1.0,2,1.0,1.0,1,1,1,1,1,1,LC


## Adding pre-survey data

We would like to use following student attributes from pre and post surveys:
* gender
* level of physics and chemistry lab experience
* level of virtual lab experience
* if they have used a similar sim before
* perception of competence and control
* task interpretation
* perceived value

In [13]:
student_data_L = pd.DataFrame({'sid':ids,'sim':'L','similar_sim':0})
student_data_C = pd.DataFrame({'sid':ids,'sim':'C','similar_sim':0})

pre_survey_df['similar_same_L'] = (pre_survey_df['similar_L'] + pre_survey_df['same_L']).replace(2,1)
pre_survey_df['similar_same_C'] = (pre_survey_df['similar_C'] + pre_survey_df['same_C']).replace(2,1)

similar_same_L = dict(zip(pre_survey_df['sid'],pre_survey_df['similar_same_L']))
similar_same_C = dict(zip(pre_survey_df['sid'],pre_survey_df['similar_same_C']))

student_data_L['similar_sim'] = student_data_L['sid'].apply(lambda sid: similar_same_L[sid])
student_data_C['similar_sim'] = student_data_C['sid'].apply(lambda sid: similar_same_C[sid])

student_data = pd.concat([student_data_L,student_data_C],axis=0).reset_index(drop=True)

In [14]:
student_data.head()

,sid,sim,similar_sim
0,11612162,L,1
1,13660166,L,1
2,41947147,L,0
3,64006159,L,0
4,15749160,L,1


In [16]:
lab_columns = ['[prior_lab] What lab courses are you presently taking or have taken in the past? Check all that apply. [First year undergraduate physics laboratory]','[prior_lab] What lab courses are you presently taking or have taken in the past? Check all that apply. [First year undergraduate chemistry laboratory]','[prior_lab] What lab courses are you presently taking or have taken in the past? Check all that apply. [Higher level chemistry labs]']
pre_survey_df['lab experience'] = pre_survey_df[lab_columns].fillna(0).sum(axis=1).replace(3,1).replace(2,1)
student_data = student_data.merge(pre_survey_df[['sid','lab experience','prior_number_virtual_labs']], how="inner", on=['sid']);
student_data = student_data.merge(attitudes_df, how="inner", on=['sid']);

In [17]:
student_data.head()

,sid,sim,similar_sim,lab experience,prior_number_virtual_labs,overall POCC,PC1,PC2
0,11612162,L,1,1.0,1,14,1.898,-1.476
1,11612162,C,0,1.0,1,14,1.898,-1.476
2,13660166,L,1,1.0,1,13,-2.233,-0.004
3,13660166,C,0,1.0,1,13,-2.233,-0.004
4,41947147,L,0,1.0,3,20,2.830,-1.208


In [ ]:
#JUST NEED TO ADD GENDER!!!

In [ ]:
pre_survey_df.columns

In [ ]:
pre_survey_df.head(5)

In [ ]:
data['level_experience_sims'] = data.apply(lambda row: pre_survey_df.loc[pre_survey_df[pre_survey_df['sid']==row['sid']].index[0],'prior_number_virtual_labs'], axis=1)
# data['experience_undergrad_labs'] = data.apply(lambda row: pre_survey_df.loc[pre_survey_df[pre_survey_df['sid']==row['sid']].index[0],'experience_undergrad_labs'], axis=1)
data['used_similar_sim_L'] = data[data['sim']=='L'].apply(lambda row: pre_survey_df.loc[pre_survey_df[pre_survey_df['sid']==row['sid']].index[0],'similar_L'],axis=1)
data['used_similar_sim_L'].fillna(0, inplace=True)
data['used_similar_sim_C'] = data[data['sim']=='C'].apply(lambda row: pre_survey_df.loc[pre_survey_df[pre_survey_df['sid']==row['sid']].index[0],'similar_C'],axis=1)
data['used_similar_sim_C'].fillna(0, inplace=True)
data['used_similar_sim'] = data['used_similar_sim_L'] + data['used_similar_sim_C']

In [ ]:
data.sort_values('sid').head(12)

In [ ]:
data.hist()

## Exporting the data

In [ ]:
# export_data = data[["sid","variable","pre","main","sim","CVS_graph","sim_index"]]
export_data = data.copy()
export_data.to_csv('dataframe_all_factors_by_student_x_variable.csv')

## Stats tools

In [ ]:
def eta_squared(aov):
    aov['eta_sq'] = 'NaN'
    aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])
    return aov
 
def omega_squared(aov):
    mse = aov['sum_sq'][-1]/aov['df'][-1]
    aov['omega_sq'] = 'NaN'
    aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*mse))/(sum(aov['sum_sq'])+mse)
    return aov

def clean_summary(model):
    m = model.summary()
    lines = m.as_text().split('\n')
    lines = [l for l in lines if 'C(sid)' not in l]
    return '\n'.join(lines)

# Results to present in methods
## Student population

In [ ]:
posts = get_all_posts_surveys()
genders = ['gender','[gender] To which gender do you most identify? [Man]','[gender] To which gender do you most identify? [Gender non conforming/non-binary]','[gender] To which gender do you most identify? [Trans*]','[gender] To which gender do you most identify? [Rather specify:]','[gender] To which gender do you most identify? [Rather specify:] [text]','[gender] To which gender do you most identify? [Prefer not to answer]']
for g in genders:
    if '?' in g:
        gender = g.split('?')[1]
    else:
        gender = ' [Woman]'
    print gender, len(set(posts[posts[g]==1]['sid'])), round(len(set(posts[posts[g]==1]['sid']))/float(N)*100,1)

## Student prior experience

In [ ]:
pd.pivot_table(data, values=['level_experience_sims'], index=['sid'],aggfunc=sum)[['level_experience_sims']].hist()
print '''where:
 0 -> None
 1 -> 1-2 (roughly)
 2 -> 3-5 (roughly)
 3 -> 6+ (roughly)'''

In [ ]:
# print  'experience in virtual labs teaching light absorbance or capacitors before'
print data[['used_similar_sim_C','used_similar_sim_L']].describe()
pd.pivot_table(data, values=['used_similar_sim_C','used_similar_sim_L'], index=['sid'],aggfunc=np.sum)[['used_similar_sim_C','used_similar_sim_L']].hist(sharey=True)

In [ ]:
pd.pivot_table(data, values=['sid'], index=['sim_index','variable'],aggfunc=len)

# Learning through inquiry
## Overall

Let's look at pre to post to see if students learn.

-> repeat stacked bar chart from worksheet data connector


### Descriptives
\# of statments per type if student CSV
(pivot table)

In [ ]:
d = pd.pivot_table(data, values=['pre','main'], index=['variable','CVS_levels_3'],aggfunc=(len,np.mean, np.std))
# e= pd.pivot_table(data, values=['pre','main'], index=['variable','CVS_graph_3'],aggfunc=(np.mean, np.std))
d

# Does using strategy help student make model?

## Using different levels of CVS and model statements

We have 10 independant variables:
* sim_index
* sim
* variable
* level_experience_sims
* experience_undergrad_labs
* used_similar_sim
* CVS_context
* has_pre_id
* has_pre_qual
* has_pre_quant

And 3 dependant:
* has_main_id
* has_main_qual
* has_main_quant


*NOTE:* we don't include variable and sim in the same model

## Using different levels of CVS and model correctness

We have 8 control variables:
* sim_index
* sim
* variable
* level_experience_sims
* experience_undergrad_labs
* used_similar_sim
* CVS_context
* pre_id_corr
* pre_qual_corr
* pre_quant_corr

and 3 dependant:
* main_id_corr
* main_qual_corr
* main_quant_corr


*NOTE:* we don't include variable and sim in the same model

We first remove students with max pre for a certain variable

In [ ]:
print data.shape
filtered_maxpre_data = data.copy()
filtered_maxpre_data = filtered_maxpre_data[filtered_maxpre_data['pre']<3]
#remove student who didn't do CVS-explore for 2 variables
print filtered_maxpre_data.shape
# print filtered_maxpre_data.describe()
filtered_maxpre_data = filtered_maxpre_data[(filtered_maxpre_data['sid']!= 11384795)|(filtered_maxpre_data['variable'].isin(['Width','Concentration']))]
# print filtered_maxpre_data[filtered_maxpre_data['sid']==11384795]
# filtered_maxpre_data = filtered_maxpre_data[(filtered_maxpre_data['sid']!= 11384795)&(filtered_maxpre_data['variable']=='Separation')]
print filtered_maxpre_data.shape
# print filtered_maxpre_data.describe()

In [ ]:
# covariates_for_interaction_model = " + sim_index + C(sid) + level_experience_sims + experience_undergrad_labs + used_similar_sim"
covariates_for_parsimonious_model = "+ variable + pre + sim_index + C(sid) + level_experience_sims + used_similar_sim"

In [ ]:
formula = 'main ~ C(CVS_levels_3) '+ covariates_for_parsimonious_model
model = ols(formula, filtered_maxpre_data).fit()
print "\n\nModel: ",formula
aov_table = anova_lm(model, typ=3)
eta_squared(aov_table)
omega_squared(aov_table)
print "\nAnova table using type 3\n"
print(aov_table)
print "\nHere is the linear model with coefficients and confidence intervals (removed stats for individual student ids):\n"
print clean_summary(model)

In [ ]:
# aov_table = aov_table.round({'F': 2,'eta_sq':2})
# print tabulate(aov_table[['F','PR(>F)','eta_sq']].sort_values('F',ascending =False),tablefmt='latex')
aov_table = aov_table.round({'F': 2, 'PR(>F)': 2,'eta_sq':2})
print tabulate(aov_table[['F','PR(>F)','eta_sq']].sort_values('F',ascending =False),tablefmt='latex')

In [ ]:
# print(model.summary().as_latex())

## Sensitivity analysis on CVS criteria using BIC

In [ ]:
# for CVS in ['CVS_levels_2','CVS_levels_3','CVS_levels_4','CVS_levels_5']:
#     formula = 'main ~ C('+CVS+')' + covariates_for_parsimonious_model
#     model = ols(formula, filtered_maxpre_data).fit()
#     print "\n\nModel: ",formula
#     aov_table = anova_lm(model, typ=3)
#     eta_squared(aov_table)
#     omega_squared(aov_table)
#     print "\nAnova table using type 3\n"
#     print(aov_table)
#     print "\nHere is the linear model with coefficients and confidence intervals (removed stats for individual student ids):\n"
#     print clean_summary(model)

These results were produced using AOV of linear regression with Type III SS

We think a threshold of 3 pts is stricks a balance between 1) measuring more deliberatness than 2 pts and 2) it is less stringent than using more points.

Here are the AIC and BIC of the models (lower means better model controlling for number of parameters)


    no interactions
    #pts	R^2
    2		0.449
    3		0.447
    4		0.443
    5		0.428

None of these models are drastically different from each other, thought 5pts is definitely the worst

We pick #pts = 3 and call it a day

In [ ]:
results = [['Threshold','Concentration ','Width ','Area ','Separation','$R^2$']]
for t,r in zip([2,3,4,5],[0.449,0.447,0.443,0.428]):
    res = [t]
    res.extend([round(sum(data[data['variable']==v]['CVS_graph_'+str(t)])/float(N),2) for v in ['Concentration','Width','Area','Separation']])
    res.append(r)
    results.append(res)
print tabulate(results,tablefmt='latex')

### Models per variable

In [ ]:
per_variable = pd.pivot_table(filtered_maxpre_data, values=['main'], index=['variable','CVS_levels_3'],aggfunc=(np.mean))
per_variable.plot(kind='bar')
print "This plot needs to be redone with standard deviation bars and organized by sim, colored by level"

In [ ]:
covariates_for_variable_models = " + pre + sim_index + level_experience_sims + used_similar_sim + use_graph"

In [ ]:
for variable in ['Width', 'Concentration','Area','Separation']:
    print "__________________________________\nFOR VARIABLE ", variable, '\n__________________________________'
    formula = 'main ~ C(CVS_levels_3)' + covariates_for_variable_models
    model = ols(formula, filtered_maxpre_data[filtered_maxpre_data['variable']==variable]).fit()
    print "\n\nModel: ",formula
    aov_table = anova_lm(model, typ=3)
    eta_squared(aov_table)
    omega_squared(aov_table)
    print "\nAnova table using type 3 errors\n"
    print(aov_table)
#     print "\nHere is the linear model with coefficients and confidence intervals:\n"
    print clean_summary(model)

### A closer look at CVS graph with inverse scale for Separation

In [ ]:
graph_inverse_cvs_df = pd.read_csv('graph_inverse_cvs_df.txt', sep='\t')
graph_inverse_cvs_df['used_inverse_separation']=graph_inverse_cvs_df['Separation']>3
graph_inverse_cvs_df['sid']=graph_inverse_cvs_df['studentid']

In [ ]:
variable ='Separation'
temp = filtered_maxpre_data[filtered_maxpre_data['variable']==variable]
separation_data = temp.merge(graph_inverse_cvs_df, on=['sid'], how='inner')
print "__________________________________\nFOR VARIABLE ", variable, '\n__________________________________'
formula = 'main ~ C(CVS_levels_3)*used_inverse_separation' + covariates_for_variable_models
model = ols(formula, separation_data).fit()
print "\n\nModel: ",formula
aov_table = anova_lm(model, typ=3)
eta_squared(aov_table)
omega_squared(aov_table)
print "\nAnova table using type 3 errors\n"
print(aov_table)
#     print "\nHere is the linear model with coefficients and confidence intervals:\n"
print clean_summary(model)

In [ ]:
separation_data[(separation_data['CVS_levels_3']==2)&(separation_data['used_inverse_separation']==False)]['main'].unique()

In [ ]:
fig,ax = plt.subplots(nrows=1,ncols=3,figsize=(15,5))
sns.countplot(data=separation_data[(separation_data['main']!=0)&(separation_data['CVS_levels_3']!=2)],y='main',ax=ax[0],label='main score',color='blue',alpha=0.7);
sns.countplot(data=separation_data[(separation_data['main']!=0)&(separation_data['CVS_levels_3']==2)&(separation_data['used_inverse_separation']==False)],y='main',ax=ax[1],label='main score',color='blue',alpha=0.7);
sns.countplot(data=separation_data[(separation_data['main']!=0)&(separation_data['CVS_levels_3']==2)&(separation_data['used_inverse_separation']==True)],y='main',ax=ax[2],label='main score',color='blue',alpha=0.7);
ax[0].set(xlabel='Number of students')
ax[1].set(xlabel='Number of students')
ax[2].set(xlabel='Number of students')
ax[0].set(ylabel='main score if didn\'t do CVS graph')
ax[1].set(ylabel='main score if did CVS graph')
ax[2].set(ylabel='main score if did CVS graph with inverse scale')
ax[0].set(yticklabels = ['Identify','Qualitative','Quantitative'])
ax[1].set(yticklabels = ['','','',''])
ax[2].set(yticklabels = ['','','',''])
ax[0].set(xlim=(0,60))
ax[1].set(xlim=(0,60))
ax[2].set(xlim=(0,60))
print "I removed the students with 'all incorrect/None' main score"

## Post-hoc analysis on levels using 3pts as threshold

In [ ]:
for ignore in [0,1,2]:
    formula = 'main ~ C(CVS_levels_3)' + covariates_for_parsimonious_model
    model = ols(formula, filtered_maxpre_data[filtered_maxpre_data[CVS]!=ignore]).fit()
    print "\n\nModel: ",formula
    print "Comparing CVS levels ignoring level ",ignore
    aov_table = anova_lm(model, typ=3)
    eta_squared(aov_table)
    omega_squared(aov_table)
    print "\nAnova table using type 3 errors\n"
    print(aov_table)
    print clean_summary(model)

In [ ]:
levels = pd.pivot_table(filtered_maxpre_data, values=['main'], index=['CVS_levels_3'],aggfunc=np.mean)
levels.plot(kind='barh')
print "This plot needs to be redone with standard deviation bars and colored by level and stars given the tests above"
print '0-1 => -'
print '1-2 => *'
print '0-2 => ***'

In [ ]:
fig,ax = plt.subplots(nrows=1,ncols=3,figsize=(12,5))
for N in [0,1,2]:
    sns.countplot(data=filtered_maxpre_data[(filtered_maxpre_data['CVS_levels_3']==N)],y='main',ax=ax[N],label=str(N),color='blue',alpha=0.7);
    ax[N].set(ylabel='')
    ax[N].set(xlabel='')
    ax[N].set(yticklabels = ['','','',''])
    ax[N].set(xlim=(0,180))
ax[0].set(ylabel='main score')
ax[1].set(xlabel='Number of students')
ax[0].set(yticklabels = ['None-all incorrect','Identify','Qualitative','Quantitative']);

In [ ]:
fig,ax = plt.subplots(nrows=1,ncols=4,figsize=(12,5))
for N,v in enumerate(['Separation',"Area",'Concentration','Width']):
    sns.countplot(data=filtered_maxpre_data[(filtered_maxpre_data['main']>1)&(filtered_maxpre_data['CVS_levels_3']==2)&(filtered_maxpre_data['variable']==v)],y='main',ax=ax[N],label=str(N),color='blue',alpha=0.7);
    ax[N].set(ylabel='')
    ax[N].set(xlabel=v)
    ax[N].set(yticklabels = ['','','',''])
    ax[N].set(xlim=(0,50))
ax[0].set(ylabel='main score')
# ax[1].set(xlabel='Number of students')
ax[0].set(yticklabels = ['Qualitative','Quantitative']);
# ["Area",'Concentration','Separation','Width']

### Post-hoc analysis on experience in physic undergraduate labs

In [ ]:
# posthoc_exp_physics_labs = pd.pivot_table(data, values=['main'], index=['experience_undergrad_labs'],aggfunc=(np.mean))
# posthoc_exp_physics_labs.plot(kind='bar')
# print "This plot needs to be redone with standard deviation bars and organized by sim, colored by level"

### Post-hoc analysis on order

In [ ]:
# posthoc_sim_index = pd.pivot_table(data, values=['main'], index=['sim_index'],aggfunc=(np.mean))
# posthoc_sim_index.plot(kind='bar')
# print "This plot needs to be redone with standard deviation bars and organized by sim, colored by level"

### Post-hoc analysis on variable

In [ ]:
# posthoc_variable = pd.pivot_table(data, values=['main'], index=['variable'],aggfunc=(np.mean))
# posthoc_variable.plot(kind='bar')
# print "This plot needs to be redone with standard deviation bars and organized by sim, colored by level"

Findings:
* CVS_levels=2 (graph) matters for all except Separation
* Pre matters for all except Width

## Repeat model for only student who use graph (85%)

In [ ]:
formula = 'main ~ C(CVS_levels_3)' + covariates_for_parsimonious_model
model = ols(formula, filtered_maxpre_data[(filtered_maxpre_data[CVS]!=0)&(filtered_maxpre_data['use_graph']==1)]).fit()
print "\n\nModel: ",formula
aov_table = anova_lm(model, typ=3)
eta_squared(aov_table)
omega_squared(aov_table)
print "\nAnova table using type 3 errors\n"
print(aov_table)
print clean_summary(model)

# What affects use CVS-like inquiry strategies?

For some reason adding students in the matric turns it into a singular matrix and fails the analysis. Probably because two students are exactly identical?

## Overall, how much do they use it?

In [ ]:
data['CVS_table'] = data['CVS_table_3']
data['CVS_graph'] = data['CVS_graph_3']
sums = pd.pivot_table(data, values=['CVS_table','CVS_graph'], index=['sid'], aggfunc=np.sum)

In [ ]:
print data.shape

In [ ]:
for threshold in ['2','3','4','5']:
    print '\nFor CVs with {0} pts as a threshold'.format(threshold)
    sums = pd.pivot_table(data, values=['CVS_table_'+threshold,'CVS_graph_'+threshold], index=['sid'], aggfunc=np.sum)
    once = len(sums[sums['CVS_table_'+threshold]>0])/float(N)*100
    all4 = len(sums[sums['CVS_table_'+threshold]==4])/float(N)*100
    print "   {0}% of students use CVS table once and {1}% do it for all variables.".format(int(once),int(all4))
    once = len(sums[sums['CVS_graph_'+threshold]>0])/float(N)*100
    all4 = len(sums[sums['CVS_graph_'+threshold]==4])/float(N)*100
    print "   {0}% of students use CVS graph once and {1}% do it for all variables.".format(int(once),int(all4))

## Are they consistent in their usage of CVS graph?

In [ ]:
sums2 = pd.pivot_table(data, values=['CVS_graph'], index=['sid','sim'], aggfunc=np.sum)
sums2 = sums2.reset_index(level=['sim','sid'])
# sums2.head()

In [ ]:
cvs_sim = np.zeros((3,3),dtype=int)
for i in range(3):
    for j in range(3):
        cvs_sim[i,j] = len(set(sums2[(sums2['sim']=='C')&(sums2['CVS_graph']==i)]['sid'])&set(sums2[(sums2['sim']=='L')&(sums2['CVS_graph']==j)]['sid']))
ax = sns.heatmap(np.flip(cvs_sim,0),annot=True, fmt="d")
ax.set_xticklabels(['0/2','1/2','2/2'])
ax.set_yticklabels(['0/2','1/2','2/2'])
plt.xlabel('Number of light absorbance variables')
plt.ylabel('Number of Caps variables')
plt.title('Map of student use of CVS in their graph for variables per simulation')

High usage is consistent. If they do CVS with 1 variable in one sim, they do it with 2 in the other (probaly second sim).We'll see how order makes a difference later.

## What affects use of CVS
### Model with interaction (killed)

In [ ]:
# formula = 'CVS_graph ~ pre*variable + sim_index*variable + pre*sim_index + level_experience_sims + experience_undergrad_labs + used_similar_sim'
# print 'model: ', formula,'\n'
# from patsy import dmatrices
# Y, X = dmatrices(formula, data, return_type = 'dataframe')
# # print X.columns
# logit = Logit(Y, X)
# model = logit.fit()
# print model.summary()
# # note that stats model has no module for running an anova on a logistic regression model

### Parismonious model

In [ ]:
formula = 'CVS_graph ~ pre + variable + sim_index +level_experience_sims + experience_undergrad_labs + used_similar_sim'
print 'model: ', formula,'\n'
from patsy import dmatrices
Y, X = dmatrices(formula, data, return_type = 'dataframe')
# print X.columns
logit = Logit(Y, X)
model = logit.fit()
print model.summary()
# aov_table = anova_lm(model)
# eta_squared(aov_table)
# omega_squared(aov_table)
# # print "\nAnova table using type 2 errors\n"
# print(aov_table)

In order of what affects CVS_graph most:
* experience in undergraduate labs
* order
* prior knowledge

Having used a similar sim doesn't matter, variable doesn't matter and sim doesn't matter.

Experience in sims mattered before we included both physics and chem experience in labs

In [ ]:
model.summary()

In [ ]:
print model.summary().as_latex()

### post hoc of experience in undergraduate labs

In [ ]:
# cvs_exp = pd.pivot_table(data, values=['CVS_graph'], index=['experience_undergrad_labs'],aggfunc=(np.mean,np.std))
# print "Here is prob that they do CVS graph depending on the experience_undergrad_labs"
# cvs_exp['CVS_graph']['mean'].plot.bar(yerr=cvs_exp['CVS_graph']['std'])

### post hoc of activity order

In [ ]:
# cvs_exp = pd.pivot_table(data, values=['CVS_graph'], index=['sim_index'],aggfunc=(np.mean,np.std))
# print "Here is prob that they do CVS graph depending on sim_index"
# cvs_exp['CVS_graph']['mean'].plot.bar(yerr=cvs_exp['CVS_graph']['std'])

### post hoc of experience with sims

In [ ]:
# cvs_exp = pd.pivot_table(data, values=['CVS_graph'], index=['level_experience_sims'],aggfunc=(np.mean,np.std))
# print "Here is prob that they do CVS graph depending on level_experience_sims"
# cvs_exp['CVS_graph']['mean'].plot.bar(yerr=cvs_exp['CVS_graph']['std'])
# print '''where:
#  0 -> None
#  1 -> 1-2 (roughly)
#  2 -> 3-5 (roughly)
#  3 -> 6+ (roughly)'''

### post hoc of pre

In [ ]:
# cvs_pre = pd.pivot_table(data, values=['CVS_graph'], index=['pre'],aggfunc=(np.mean,np.std))
# print "Here is the avg pre score of students depending if they do CVS graph or not"
# cvs_pre['CVS_graph']['mean'].plot.bar(yerr=cvs_pre['CVS_graph']['std'])